In [ ]:
from metaflow import FlowSpec, step
import sys
import os
from pathlib import Path

path = Path(os.getcwd())
sys.path.append(str(path.parent))

import nlopt_z_match
import numpy as np
import math
import pandas as pd
import imp
import time
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import logging_mod
import os


def bar_chart(n,o,filename):
    x_in = np.arange(0,len(n))
    plot_height = int(500*math.log(len(n)))
    trace1 = go.Bar(
    y=x_in,
    x=n,
    name='Nlopt results',
    orientation = 'h',
    marker = dict(
        color = 'rgba(158,202,225, 0.6)',
        line = dict(
            color = 'rgba(246, 78, 139, 1.0)',
            width = 3)
    )
    )

    trace2 = go.Bar(
    y=x_in,
    x=o,
    name='Original weights',
    orientation = 'h',
    marker = dict(
        color = 'rgba(58, 71, 80, 0.6)',
        line = dict(
            color = 'rgba(58, 71, 80, 1.0)',
            width = 3)
    ),
    opacity = 0.7
    )

    data = [trace1, trace2]
    layout = go.Layout(
    barmode='overlay',
    height = plot_height
    )

    fig = go.Figure(data=data, layout=layout)
    plot(fig, filename=filename, auto_open=False)


def get_weights_animal1():
    weights = [1]*72
    weights[15] = 9
    weights[2] = 7
    weights[5] = 5
    weights[34] = 8
    weights[10] = 10
    weights[25] = 4.5
    weights[35] = 4.5
    weights[45] = 17.78
    weights[55] = 2.0
    return(weights)

def get_weights_animal2():
    weights = [1]*72
    weights[1] = 15
    weights[2] = 5
    weights[5] = 7
    weights[34] = 8
    weights[10] = 2
    weights[25] = 10
    weights[35] = 7
    weights[45] = 15
    weights[55] = 5.0
    return(weights)

def get_weights_big1():
    weights = [1]*30
    weights[15] = 9
    weights[2] = 7
    weights[5] = 5
    weights[28] = 8
    weights[10] = 10
    weights[25] = 4.5
    weights[3] = 4.5
    weights[4] = 17.78
    weights[5] = 2.0
    return(weights)

def get_weights_big2():
    weights = [1]*30
    weights[10] = 2
    weights[1] = 7.5
    weights[5] = 5
    weights[20] = 8
    weights[10] = 9
    weights[22] = 15
    weights[12] = 4.5
    weights[18] = 20
    weights[17] = 2.0
    return(weights)


class LinearFlow(FlowSpec):
    @step
    def start(self):
        self.cases = ['animal_case1_finite','animal_case1_svd','big_case1_finite','big_case1_svd']
        for case in self.cases:
            if(os.path.exists('./' + case) == False):
                os.mkdir('./' + case)
        self.next(self.animal_case1_finite)


    @step
    def animal_case1_finite(self):
        self.title = 'animal_case1_finite'
        self.filename = '../Animal_Data_study_backup.csv'
        weights = get_weights_animal1()
        self.weights_array = weights
        self.grad_calculation = 'finite'
        self.imds_init_flag = 0

        print("Starting Backward MDS --- case 1 -- finite")
        self.call_nlopt()
        self.next(self.plot)


    @step
    def plot(self):
        bar_chart(self.nlopt_weights, self.orig, self.title)
        self.move_files()
        self.next(self.animal_case1_svd)


    @step
    def animal_case1_svd(self):
        self.title = 'animal_case1_svd'
        self.filename = '../Animal_Data_study_backup.csv'
        weights = get_weights_animal1()
        self.weights_array = weights
        self.grad_calculation = 'svd'
        self.imds_init_flag = 0

        print("Starting Backward MDS --- case 1 -- svd ")
        self.call_nlopt()
        self.next(self.plot2)


    @step
    def plot2(self):
        bar_chart(self.nlopt_weights, self.orig, self.title)
        self.move_files()
        self.next(self.big_case1_finite)

    # ------------------------------- Bigger data -------------------------- #

    @step
    def big_case1_finite(self):
        self.title = 'big_case1_finite'
        self.filename = '../Large_data/wdbc_cleaned.csv'
        weights = get_weights_big1()
        self.weights_array = weights
        self.grad_calculation = 'finite'
        self.imds_init_flag = 0

        print("Starting Backward MDS large data --- case 1 -- finite")
        self.call_nlopt()
        self.next(self.plot3)


    @step
    def plot3(self):
        bar_chart(self.nlopt_weights, self.orig, self.title)
        self.move_files()
        self.next(self.big_case1_svd)


    @step
    def big_case1_svd(self):
        self.title = 'big_case1_svd'
        self.filename = '../Large_data/wdbc_cleaned.csv'
        weights = get_weights_big1()
        self.weights_array = weights
        self.grad_calculation = 'svd'
        self.imds_init_flag = 0

        print("Starting Backward MDS large data --- case 1 -- svd ")
        self.call_nlopt()
        self.next(self.plot4)


    @step
    def plot4(self):
        bar_chart(self.nlopt_weights, self.orig, self.title)
        self.move_files()
        self.next(self.end)


    def call_nlopt(self):
        return_object =  nlopt_z_match.main(self.filename,
                                            self.weights_array,
                                            self.grad_calculation)
        self.nlopt_weights = return_object['nlopt_weights']
        self.orig = return_object['weights']
        self.nlopt_residual = return_object['nlopt_res']
        self.fn_count = return_object['fn_cnt']
        self.time_elapsed = return_object['time_elapsed']
        self.convergence = return_object['convergence']
        self.time_per_call = self.time_elapsed / self.fn_count
        print('The result is: %s' % self.nlopt_residual, self.fn_count, self.time_elapsed)


    def move_files(self):
        move_string = "mv Results.out " + self.title + "/"
        os.system(move_string)
        move_string = "mv " + self.title + ".html " + self.title + "/"
        os.system(move_string)


    @step
    def end(self):
        print("Finished")


class LinearFlow2(FlowSpec):
    @step
    def start(self):
        self.cases = ['animal_case2_finite','animal_case2_svd','big_case2_finite','big_case2_svd']
        for case in self.cases:
            if(os.path.exists('./' + case) == False):
                os.mkdir('./' + case)
        self.next(self.animal_case2_finite)


    @step
    def animal_case2_finite(self):
        self.title = 'animal_case2_finite'
        self.filename = '../Animal_Data_study_backup.csv'
        weights = get_weights_animal2()
        self.weights_array = weights
        self.grad_calculation = 'finite'
        self.imds_init_flag = 0

        print("Starting Backward MDS --- case 2 -- finite")
        self.call_nlopt()
        self.next(self.plot)


    @step
    def plot(self):
        bar_chart(self.nlopt_weights, self.orig, self.title)
        self.move_files()
        self.next(self.animal_case2_svd)


    @step
    def animal_case2_svd(self):
        self.title = 'animal_case2_svd'
        self.filename = '../Animal_Data_study_backup.csv'
        weights = get_weights_animal2()
        self.weights_array = weights
        self.grad_calculation = 'svd'
        self.imds_init_flag = 0

        print("Starting Backward MDS --- case 2 -- svd ")
        self.call_nlopt()
        self.next(self.plot2)


    @step
    def plot2(self):
        bar_chart(self.nlopt_weights, self.orig, self.title)
        self.move_files()
        self.next(self.big_case2_finite)

    # ------------------------------- Bigger data -------------------------- #

    @step
    def big_case2_finite(self):
        self.title = 'big_case2_finite'
        self.filename = '../Large_data/wdbc_cleaned.csv'
        weights = get_weights_big2()
        self.weights_array = weights
        self.grad_calculation = 'finite'
        self.imds_init_flag = 0

        print("Starting Backward MDS large data --- case 2 -- finite")
        self.call_nlopt()
        self.next(self.plot3)


    @step
    def plot3(self):
        bar_chart(self.nlopt_weights, self.orig, self.title)
        self.move_files()
        self.next(self.big_case2_svd)


    @step
    def big_case2_svd(self):
        self.title = 'big_case2_svd'
        self.filename = '../Large_data/wdbc_cleaned.csv'
        weights = get_weights_big2()
        self.weights_array = weights
        self.grad_calculation = 'svd'
        self.imds_init_flag = 0

        print("Starting Backward MDS large data --- case 2 -- svd ")
        self.call_nlopt()
        self.next(self.plot4)


    @step
    def plot4(self):
        bar_chart(self.nlopt_weights, self.orig, self.title)
        self.move_files()
        self.next(self.end)


    def call_nlopt(self):
        return_object =  nlopt_z_match.main(self.filename,
                                            self.weights_array,
                                            self.grad_calculation)
        self.nlopt_weights = return_object['nlopt_weights']
        self.orig = return_object['weights']
        self.nlopt_residual = return_object['nlopt_res']
        self.fn_count = return_object['fn_cnt']
        self.time_elapsed = return_object['time_elapsed']
        self.convergence = return_object['convergence']
        self.time_per_call = self.time_elapsed / self.fn_count
        print('The result is: %s' % self.nlopt_residual, self.fn_count, self.time_elapsed)


    def move_files(self):
        move_string = "mv Results.out " + self.title + "/"
        os.system(move_string)
        move_string = "mv " + self.title + ".html " + self.title + "/"
        os.system(move_string)


    @step
    def end(self):
        print("Finished")


if __name__ == '__main__':
    LinearFlow2()
